In [ ]:
import tensorflow as tf
import os

In [ ]:
_URL = "http://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin = _URL, fname= " flower_photos.tgz", extract=True, cache_subdir="/content")
base_dir = os.path.join(os.path.dirname(zip_file), "flower_photos")

228827136/228813984 [==============================] - 1s 0us/step


In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64 

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2

)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = "training"
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = "validation"
)



Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
labels ="\n".join(sorted(train_generator.class_indices.keys())) 

with open("labels.txt", "w") as f:
  f.write(labels)   

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape= IMG_SHAPE, include_top= False, weights= "imagenet")

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation="softmax")

])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
epochs = 10 

history = model.fit(
    train_generator,
    epochs = epochs,
    validation_data = val_generator
    )

Epoch 1/10
46/46 [==============================] - 124s 3s/step - loss: 0.8335 - accuracy: 0.7383 - val_loss: 0.4943 - val_accuracy: 0.8331
Epoch 2/10
46/46 [==============================] - 119s 3s/step - loss: 0.3201 - accuracy: 0.8850 - val_loss: 0.4220 - val_accuracy: 0.8564
Epoch 3/10
46/46 [==============================] - 136s 3s/step - loss: 0.1990 - accuracy: 0.9316 - val_loss: 0.4220 - val_accuracy: 0.8646
Epoch 4/10
46/46 [==============================] - 119s 3s/step - loss: 0.1213 - accuracy: 0.9599 - val_loss: 0.4620 - val_accuracy: 0.8577
Epoch 5/10
46/46 [==============================] - 117s 3s/step - loss: 0.0850 - accuracy: 0.9741 - val_loss: 0.4822 - val_accuracy: 0.8550
Epoch 6/10
46/46 [==============================] - 135s 3s/step - loss: 0.0528 - accuracy: 0.9874 - val_loss: 0.4962 - val_accuracy: 0.8495
Epoch 7/10
46/46 [==============================] - 116s 3s/step - loss: 0.0441 - accuracy: 0.9901 - val_loss: 0.5852 - val_accuracy: 0.8509
Epoch 8/10
46

In [ ]:
saved_model_dir = ""
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open("model.tflite","wb") as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmp3oz7ge2l/assets


INFO:tensorflow:Assets written to: /tmp/tmp3oz7ge2l/assets


In [ ]:
from google.colab import files

files.download("model.tflite")
files.download("labels.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>